## Часть 1
Напишите скрипт, который скачивает все данные прошедших президентских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227). Затем нужно перейти на сайты региональных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название региона
- название ТИК
- номер УИК
- 20 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

### Часть 2
Используя скаченные данные и `Pandas` (или `PySpark`) посчитайте:
- явку (%) по всем регионам, результат отсортировать по убыванию
- выберите произвольного кандидата и найдите тот избиратльный участок, на котором он получил наибольший результат  (%, учитывать участки на которых проголосовало больше 300 человек)
- найдите регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна
- посчитайте дисперсию по явке для каждого региона (учитывать УИК)
- для каждого кандидата посчитайте таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [4]:
import lxml.html
import lxml.etree
import requests
import pandas as pd
import re
import numpy as np

In [5]:
url = 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'

In [6]:
response = requests.get(url)
tree = lxml.html.fromstring(response.text)

In [7]:
urles = tree.xpath('//table//tr//td//nobr/a/@href')

cols = tree.xpath('//td[2]/nobr/text()')

frames = []

for url in urles:
    response = requests.get(url)
    region = lxml.html.fromstring(response.text)
    region_name = region.xpath('//td[2]/text()')[0]
    urles_com = region.xpath('//nobr/a/@href')
    for url_com in urles_com:
        response = requests.get(url_com)
        commission = lxml.html.fromstring(response.text)
        url_com = commission.xpath('//tr[2]/td/a/@href')[0]
        response = requests.get(url_com)
        commission = lxml.html.fromstring(response.text)
        commission_name = commission.xpath('//table[3]/tr[2]/td[2]/text()')[0]
        uiks_n = len(commission.xpath('//td[2]/div/table/tr[1]/td/nobr'))
        values = commission.xpath('//div/table/tr/td/nobr/b/text()')
        table = {col: values[uiks_n*i:uiks_n*(i+1)] for i, col in enumerate(cols)}
        table['Номер УИК'] = np.array([i+1 for i in range(0, uiks_n)])
        table['Регион'] = region_name
        table['ТИК'] = commission_name
        
        frames.append(pd.DataFrame(table))


In [8]:
df = pd.concat(frames)

In [9]:
df.to_csv('election.csv', encoding='utf-8', index=False)

In [24]:
for region in df['Регион'].unique():
    presence[region] = df[df['Регион'] == region][[m1_col, m2_col]].sum().sum() / df[df['Регион'] == region][n_col].sum()
presence_df = pd.Series(presence)

In [36]:
presence_df.sort_values(ascending=False, inplace=True)
presence_df[:10]

Республика Тыва                        0.936254
Ямало-Ненецкий автономный округ        0.918721
Кабардино-Балкарская Республика        0.916979
Чеченская Республика                   0.915093
Республика Северная Осетия - Алания    0.899370
Республика Дагестан                    0.874435
Карачаево-Черкесская Республика        0.873438
Кемеровская область                    0.830756
Чукотский автономный округ             0.822605
Республика Ингушетия                   0.819594
dtype: float64

In [38]:
for i, col in enumerate(df.columns):
    print(i, col)

0 Бабурин Сергей Николаевич
1 Грудинин Павел Николаевич
2 Жириновский Владимир Вольфович
3 Номер УИК
4 Путин Владимир Владимирович
5 Регион
6 Собчак Ксения Анатольевна
7 Сурайкин Максим Александрович
8 ТИК
9 Титов Борис Юрьевич
10 Число бюллетеней в стационарных ящиках для голосования
11 Число действительных избирательных бюллетеней
12 Число избирателей, включенных в список избирателей 
13 Число избирательных бюллетеней в переносных ящиках для голосования
14 Число избирательных бюллетеней, выданных в помещении для голосования в день голосования
15 Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования
16 Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно
17 Число избирательных бюллетеней, не учтенных при получении 
18 Число избирательных бюллетеней, полученных участковой избирательной комиссией
19 Число недействительных избирательных бюллетеней
20 Число погашенных избирательных бюллетеней
21 Число утраченных избирательных б

Рассмотрим кандидата Бабурина Сергея Николаевича. Найдем тот избирательный участок, на котором он получил наибольший результат (%, учитывая только участки, на которых проголосовало больше 300 человек):

In [51]:
candidates = ['Бабурин Сергей Николаевич', 'Грудинин Павел Николаевич', 'Жириновский Владимир Вольфович', 
              'Путин Владимир Владимирович', 'Собчак Ксения Анатольевна', 'Сурайкин Максим Александрович', 
              'Титов Борис Юрьевич', 'Явлинский Григорий Алексеевич']
candidate = 'Титов Борис Юрьевич'

In [ ]:
max(df[candidate]/df[candidates].sum(axis=1))

In [44]:
df[candidates].head()

,Бабурин Сергей Николаевич,Грудинин Павел Николаевич,Жириновский Владимир Вольфович,Путин Владимир Владимирович,Собчак Ксения Анатольевна,Сурайкин Максим Александрович,Титов Борис Юрьевич,Явлинский Григорий Алексеевич
0,0,137,32,1977,14,0,1,5
1,15,86,65,2389,13,5,6,15
2,1,62,13,2645,6,3,4,0
3,5,288,12,1642,21,6,2,2
4,2,44,6,624,6,0,0,2


In [48]:
df[df[candidate] > 300][candidate]/df[candidates].sum(axis=1)

0        0.000000
1        0.005783
2        0.000366
3        0.002528
4        0.002924
5        0.000981
6        0.004564
7        0.004274
8        0.004065
9        0.006873
10       0.015858
11       0.034672
12       0.013146
13       0.003429
14       0.007519
15       0.002381
16       0.009434
17       0.000000
18       0.008939
19       0.002549
20       0.006887
21       0.004535
22       0.004306
23       0.006502
24       0.007353
25       0.008696
26       0.000000
27       0.022876
28       0.036731
29       0.009259
           ...   
97265    0.003724
97266    0.001586
97267    0.001730
97268    0.003844
97269    0.003384
97270    0.004008
97271    0.005802
97272    0.000361
97273    0.009662
97274    0.002162
97275    0.003096
97276    0.003388
97277    0.006006
97278    0.002625
97279    0.000000
97280    0.002053
97281    0.001805
97282    0.002103
97283    0.000914
97284    0.001753
97285    0.000821
97286    0.000000
97287    0.001741
97288    0.000812
97289    0

In [52]:
df[df[candidate] > 300]

,Бабурин Сергей Николаевич,Грудинин Павел Николаевич,Жириновский Владимир Вольфович,Номер УИК,Путин Владимир Владимирович,Регион,Собчак Ксения Анатольевна,Сурайкин Максим Александрович,ТИК,Титов Борис Юрьевич,...,Число избирательных бюллетеней в переносных ящиках для голосования,"Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, не учтенных при получении","Число избирательных бюллетеней, полученных участковой избирательной комиссией",Число недействительных избирательных бюллетеней,Число погашенных избирательных бюллетеней,Число утраченных избирательных бюллетеней,Явлинский Григорий Алексеевич
6676,0,14,0,31,1376,Республика Дагестан,0,0,Шамильская,330,...,271,1450,271,0,0,1784,1,63,0,0
